## FULL NOTEBOOK — NLP → NLU → NLG Pipeline with Matthew (KJV)

### SETUP — Downloading Matthew (KJV) from Project Gutenberg

In [1]:
import requests
import re
import numpy as np
from collections import Counter, defaultdict
from typing import List, Dict, Tuple
import json

# Download King James Bible from Project Gutenberg
print("=" * 80)
print("DATA ACQUISITION")
print("=" * 80)

url = "https://www.gutenberg.org/cache/epub/10/pg10.txt"
response = requests.get(url)
text = response.text

# Extract only Matthew
start = text.find("1:1 The book of the generation of Jesus Christ, the son of David")
end = text.find("The Gospel According to Saint Mark"+"\n1:1")
matthew_text = text[start:end]

print(f"✓ Downloaded Bible text")
print(f"✓ Extracted Gospel of Matthew: {len(matthew_text)} characters")
print(f"\nFirst 300 characters:\n{matthew_text[:300]}")

DATA ACQUISITION
✓ Downloaded Bible text
✓ Extracted Gospel of Matthew: 1039833 characters

First 300 characters:
1:1 The book of the generation of Jesus Christ, the son of David, the
son of Abraham.

1:2 Abraham begat Isaac; and Isaac begat Jacob; and Jacob begat Judas
and his brethren; 1:3 And Judas begat Phares and Zara of Thamar; and
Phares begat Esrom; and Esrom begat Aram; 1:4 And Aram begat Aminadab


### Text Processing, Cleaning & Normalization

The primary goal is to standardize the text by eliminating noise.

**Normalization:** It enforces a defined set of allowed characters. Any character not on the list (e.g., emojis, mathematical symbols, copyright symbols, currency signs, etc.) is removed.

**Delimiter Preservation:** By replacing forbidden characters with a space instead of just deleting them, you prevent words that were separated by a symbol (e.g., "word1@word2") from merging into a single, nonsensical word ("word1word2"). The result becomes "word1 word2", preserving the word boundary.

In essence, this is a high-pass filter that keeps only essential alphanumeric and common separating characters, ensuring the downstream machine learning model receives clean, predictable input.

In [4]:
import re

clean_text = matthew_text

# Lowercasing
clean_text = clean_text.lower()

# Removing Project Gutenberg headers, numbers, etc.
clean_text = re.sub(r'\n+', '\n', clean_text)          # normalize newlines
clean_text = re.sub(r'[^a-zA-Z0-9\s\:\;\,\.\?\!]', ' ', clean_text)  # keep basic punctuation
clean_text = re.sub(r'\s+', ' ', clean_text).strip()

print(clean_text[:500])

1:1 the book of the generation of jesus christ, the son of david, the son of abraham. 1:2 abraham begat isaac; and isaac begat jacob; and jacob begat judas and his brethren; 1:3 and judas begat phares and zara of thamar; and phares begat esrom; and esrom begat aram; 1:4 and aram begat aminadab; and aminadab begat naasson; and naasson begat salmon; 1:5 and salmon begat booz of rachab; and booz begat obed of ruth; and obed begat jesse; 1:6 and jesse begat david the king; and david the king begat s


### Text Splitting (sentences / paragraphs / chunks)

The process of Text Splitting is fundamental in preparing data for Large Language Models (LLMs) and Retrieval-Augmented Generation (RAG) systems. It involves breaking down a large document into smaller, more manageable, and contextually rich segments.

1. Sentence Tokenization (Splitting into Smallest Units)

The initial step uses NLTK's sent_tokenize function, which relies on the pre-trained **punkt model** to accurately identify and separate individual sentences. This is critical because sentences form the most basic units of semantic meaning.

- **Code:** sentences = nltk.sent_tokenize(clean_text)
- **Purpose:** To create a list of sentences, overcoming challenges like abbreviations (Mr., etc.) and decimal points that simple period splitting cannot handle.

2. Semantic Chunking (Grouping for Context)

The custom chunk_text function performs semantic chunking. The goal is to group several consecutive sentences into a single "chunk" that is contextually cohesive, while adhering to a defined maximum size (e.g., 150 tokens).

- **Constraint:** The chunk size must fit within the context window limit of the LLM or the input size limit for the vector embedding model.

- **Logic:** The function iteratively adds sentences to a current chunk. Before adding a new sentence, it checks the length of the potential new chunk. If the length exceeds the max_tokens limit, the current chunk is finalized, and the new sentence starts the next chunk.

- **Result:** The final chunks list contains optimized text segments ready for the next stage of the RAG pipeline (e.g., vector embedding).


In [5]:
import nltk # Import the Natural Language Toolkit library for text processing
nltk.download('punkt') # Download the 'punkt' resource for sentence tokenization
nltk.download('punkt_tab') # Download 'punkt_tab' (often used for tabular data, ensuring full sentence boundary detection)

# Use NLTK's sentence tokenizer to split the pre-cleaned text into a list of sentences
sentences = nltk.sent_tokenize(clean_text)

# Chunking for RAG-style ingestion
def chunk_text(sentences, max_tokens=200):
    """
    Groups sentences into larger chunks based on a maximum token limit,
    preserving semantic context for RAG systems.
    """
    chunks = [] # List to store the final, processed text chunks
    current = [] # List to hold sentences for the chunk currently being built

    # Iterate through each sentence
    for sent in sentences:
        # Check if adding the new sentence ('sent') exceeds the token limit
        # 1. Joins the current sentences and the new one.
        # 2. Splits by space to count words/tokens.
        if len(" ".join(current + [sent]).split()) > max_tokens:
            # Limit exceeded: finalize the current chunk
            chunks.append(" ".join(current))
            # Start a new chunk with the sentence that exceeded the limit
            current = [sent]
        else:
            # Limit not exceeded: add the sentence to the current chunk
            current.append(sent)

    # After the loop, check if there are any remaining sentences in 'current'
    if current:
        # Finalize and add the last chunk
        chunks.append(" ".join(current))

    return chunks

# Execute the chunking function with a maximum of 150 tokens per chunk
chunks = chunk_text(sentences, max_tokens=150)

# Output results for verification and analysis
print("Sentences:", len(sentences)) # Display the total number of individual sentences
print("Chunks:", len(chunks)) # Display the total number of chunks created
print(sentences[0][:400]) # Print the first 400 characters of the very first sentence

print("chunck 1:",chunks[0], "| lenght: ", len(chunks[0])) # Print the first characters of the very first chunk (should contain multiple sentences)
print("chunck 2:",chunks[1], "| lenght: ", len(chunks[1])) # Print the second characters of the very first chunk (should contain multiple sentences)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Sentences: 7610
Chunks: 1441
1:1 the book of the generation of jesus christ, the son of david, the son of abraham.
chunck 1: 1:1 the book of the generation of jesus christ, the son of david, the son of abraham. | lenght:  85
chunck 2: 1:2 abraham begat isaac; and isaac begat jacob; and jacob begat judas and his brethren; 1:3 and judas begat phares and zara of thamar; and phares begat esrom; and esrom begat aram; 1:4 and aram begat aminadab; and aminadab begat naasson; and naasson begat salmon; 1:5 and salmon begat booz of rachab; and booz begat obed of ruth; and obed begat jesse; 1:6 and jesse begat david the king; and david the king begat solomon of her that had been the wife of urias; 1:7 and solomon begat roboam; and roboam begat abia; and abia begat asa; 1:8 and asa begat josaphat; and josaphat begat joram; and joram begat ozias; 1:9 and ozias begat joatham; and joatham begat achaz; and achaz begat ezekias; 1:10 and ezekias begat manasses; and manasses begat amon; and amon begat jo

Why the Chunk Lengths are Different?

1. The Short First Sentence
The first chunk is extremely short because your sentence tokenizer (nltk.sent_tokenize) identified the first line of the input text as a single, complete sentence:

- Sentence 1: 1:1 the book of the generation of jesus christ, the son of david, the son of abraham.
- Chunk 1 is made only of Sentence 1.
- Length: 85 characters.

Because this sentence is very short, the token counter immediately checked the next sentence (Sentence 2) against the max_tokens=150 limit.

2. The Long Second Chunk (Cumulative Logic)

The second chunk is long because it contains a long list of subsequent sentences that never caused the token limit to be exceeded until the very end.

Let's look at the logic inside your chunk_text function:

```
if len(" ".join(current + [sent]).split()) > max_tokens: # The critical check
```

- **Start of Chunk 2:** Sentence 2 starts the new chunk (current = [sent]).
- **Token Counting:** Your function counts the length in **tokens (words)**, **not characters**. The **max_tokens was set to 150**.
- **The Chain of Begats:** The subsequent verses (1:2 through 1:16) in the genealogy are very short, repetitive sentences (e.g., "Abraham begat Isaac;"). Each sentence only adds a few tokens.
- **Accumulation:** The chunking function kept adding these short sentences because the cumulative word count remained under the 150 token limit. The function continues to append sentences until adding the next one pushes the total token count (not character count) past 150.
**Final Result:** The 1304 characters in Chunk 2 **represent the large number of sentences that could all fit within the 150-token** budget when grouped together.

### Tokenization

Tokenization is a fundamental preprocessing step in LLM workflows that converts raw text into a sequence of numerical IDs that a model can process. This process is essential because LLMs operate exclusively on numbers.

Tokenization

Tokenization is a fundamental preprocessing step in LLM workflows that converts raw text into a sequence of numerical IDs that a model can process. This process is essential because LLMs operate exclusively on numbers.


1. The Tokenizer
The Tokenizer is a specific piece of software (**often trained alongside the model**) that dictates how the input text should be broken down. Modern LLMs typically use sophisticated methods like **Byte Pair Encoding (BPE)** or **WordPiece** (used by BERT) because they can:

- Handle rare or misspelled words by breaking them into known sub-word units (e.g., "tokenization" might become "token" + "i" + "zation").

- Keep the overall vocabulary size manageable, which is crucial for efficient training and inference.

2. The Vocabulary
The tokenizer uses a **Vocabulary** (or Vocab) file, which is a list that maps every possible sub-word unit (token) to a unique numerical ID.

3. The Output
Tokenization produces three primary outputs, often formatted as a dictionary:

Output Key |	Description |
| :--- | :--- |
input_ids | The sequence of numerical IDs corresponding to the tokens in the text. This is the main input the model uses.
token_type_ids | Identifiers used when inputting two sentences (e.g., for Question Answering) to distinguish between the first and second sentence.
attention_mask | A binary mask used to tell the model which tokens are actual content (value 1) and which are just padding tokens (value 0).

In [6]:
from transformers import AutoTokenizer

# Loads the pre-trained tokenizer associated with the BERT model (uncased version)
# This downloads the vocabulary file and the specific tokenization rules (WordPiece).
tokenizer = AutoTokenizer.from_pretrained("bert-base-uncased")

# Encodes the first 500 characters of the clean text.
# The tokenizer automatically handles common steps like lowercasing, adding special tokens, and generating the mask.
encoded = tokenizer(clean_text[:500])

# --- Output Analysis ---
print("Token IDs:", encoded["input_ids"][:20])
# This is the list of numerical IDs the model sees. Note the special tokens:
# 101 ([CLS]) at the start and 102 ([SEP]) likely at the end of the full sequence.

print("Tokens:", tokenizer.convert_ids_to_tokens(encoded["input_ids"][:20]))
# This converts the numerical IDs back into their readable sub-word units,
# showing exactly how the text was segmented.

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Token IDs: [101, 1015, 1024, 1015, 1996, 2338, 1997, 1996, 4245, 1997, 4441, 4828, 1010, 1996, 2365, 1997, 2585, 1010, 1996, 2365]
Tokens: ['[CLS]', '1', ':', '1', 'the', 'book', 'of', 'the', 'generation', 'of', 'jesus', 'christ', ',', 'the', 'son', 'of', 'david', ',', 'the', 'son']


### Embeddings & Vectors

**Embeddings** and **Vectors** are the core mechanism that allows Large Language Models (LLMs) and systems like RAG to understand and process the semantic meaning of text. They convert text into a geometric space where meaning is represented by numerical proximity.

**Embeddings: Representing Meaning Numerically**
An embedding is a dense, low-dimensional vector representation of text (words, sentences, or chunks). The key principle is:

- **Semantic Proximity:** Text snippets that have similar meanings are mapped to vectors that are close to each other in the vector space.

- **Vector:** The term "vector" simply refers to the **list of floating-point numbers that constitute the embedding**. A typical embedding vector might have hundreds or thousands of dimensions (e.g., 384 for all-MiniLM-L6-v2).


**How Embeddings are Created**

Embeddings are generated by specialized neural network models (like the **SentenceTransformer** used in the code). These models are trained to map human language into this numerical space, capturing nuances like context, sentiment, and relationships.

In a RAG system, every chunk of your document is converted into a vector and stored in a **Vector Database**. When a user submits a query, the query is also converted into a vector. The system then finds the document vectors (chunks) that are numerically closest to the query vector, retrieving the most relevant context.


In [7]:
from sentence_transformers import SentenceTransformer

# Load a pre-trained sentence embedding model.
# "all-MiniLM-L6-v2" is a highly efficient model known for good performance
# while being very fast and producing relatively small (384-dimensional) vectors.
model = SentenceTransformer("all-MiniLM-L6-v2")

# The crucial step: encodes the list of text chunks (the 'chunks' variable from the previous step)
# into a matrix of dense vectors. Each chunk now has a numerical representation.
chunk_embeddings = model.encode(chunks)

# --- Output Analysis ---
print("Embedding dimension:", len(chunk_embeddings[0]))
# Displays the size of the vector. For all-MiniLM-L6-v2, this will be 384.

print("First vector sample:", chunk_embeddings[0][:10])
# Displays the first 10 floating-point numbers of the first chunk's vector.
# This confirms the text has been successfully converted into a numerical format,
# which is the input required for a vector database.

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Embedding dimension: 384
First vector sample: [-0.03238389  0.12138103  0.01086469  0.03534726 -0.03559428 -0.0061181
  0.02273921 -0.07431106  0.02023728  0.0783499 ]


### Ingestion Into a Vector Store (FAISS)

The final step in preparing your data for RAG is to store the numerical embeddings in a Vector Store. This specialized database is optimized for extremely fast similarity searches, allowing the system to quickly find the most relevant chunks when a user submits a query.

Core Concepts: Vector Stores

A **Vector Store** (or Vector Database) is a system designed to efficiently store and query high-dimensional vectors. Instead of searching by keyword (like traditional databases), it searches by vector distance (similarity).

- **FAISS (Facebook AI Similarity Search):** This is not a full-fledged database but a highly efficient library for clustering and searching dense vectors. It is often used for proof-of-concept RAG systems or when the vector data can fit into memory.

- **Indexing:** The process of taking the raw vectors and structuring them in a way that makes searching fast. The code uses a specific type of index: IndexFlatL2:

  - **L2 Distance:** Refers to the Euclidean distance (the straight-line distance) between two vectors. IndexFlatL2 stores the vectors exactly as they are (it's "flat") and performs exhaustive, accurate searches based on this distance.


In [13]:
!pip install faiss-cpu==1.13.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.6/23.6 MB 72.3 MB/s eta 0:00:00


In [14]:
import faiss # Import the FAISS library
import numpy as np # Import NumPy for efficient array handling

# Convert the list of chunk embeddings (from the previous step) into a NumPy array.
# FAISS requires input data to be of the float32 data type.
emb_matrix = np.array(chunk_embeddings).astype("float32")

# Get the dimension of the vectors (e.g., 384 for MiniLM).
# This is crucial for initializing the FAISS index.
dimension = emb_matrix.shape[1]

# Initialize the FAISS index.
# IndexFlatL2 means the index will store all vectors flatly and use L2 (Euclidean) distance
# for searches, providing high accuracy.
index = faiss.IndexFlatL2(dimension)

# Add all the chunk embeddings (the entire matrix) to the FAISS index.
index.add(emb_matrix)

# --- Output Analysis ---
print("Number of vectors in FAISS:", index.ntotal)
# The output confirms how many vectors (which equals the number of chunks) were successfully stored in the index.
# This FAISS index is now ready to receive a user query vector for similarity search.

Number of vectors in FAISS: 1441


### NLU — UNDERSTANDING MEANING

**Natural Language Understanding (NLU)** is a subfield of NLP and AI focused on enabling machines to comprehend and interpret the meaning of human language input. While an LLM's architecture provides a deep form of NLU inherently, these principles are often applied using specialized toolkits (like spaCy or NLTK) to extract specific, structured data from text, a process critical for building robust applications like chatbots, virtual assistants, and information retrieval systems.

#### Key NLU Tasks
NLU breaks down language analysis into several crucial tasks:

Task|	Description |	Example |
| :--- | :--- | :--- |
**Tokenization** | Breaking text into words, subwords, or punctuation. | "The car" $\rightarrow$ [The, car]
**Part-of-Speech (POS) Tagging** | Identifying the grammatical role of each word. | "dog" $\rightarrow$ **NOUN**, "ran" $\rightarrow$ **VERB**
**Named Entity Recognition (NER)** | Identifying and classifying real-world objects (people, places, organizations) in the text. | "Apple" $\rightarrow$ **ORG**, "Paris" $\rightarrow$ **GPE** (Geopolitical Entity)
**Dependency Parsing** | Analyzing the grammatical relationships between words to determine the sentence structure. |	Showing that the main subject relates to the main verb.
**Sentiment Analysis** | Determining the emotional tone (positive, negative, neutral) expressed in the text. | "That was awful." $\rightarrow$ **NEGATIVE**
**Intent Recognition** | Identifying the user's goal or purpose behind an utterance (critical for dialogue systems). | "Book a flight to Rome." $\rightarrow$ **BOOK_FLIGHT**

In [15]:
import spacy # Import the spaCy library for advanced NLP

# Load the small, English-language model (en_core_web_sm).
# This model includes pre-trained components for tasks like NER and POS tagging.
nlp = spacy.load("en_core_web_sm")

sample = "jesus went to galilee and taught in synagogues."

# Process the sample text through the spaCy pipeline.
# This generates a 'Doc' object containing all the linguistic annotations.
doc = nlp(sample)

print("Entities:")
# Extract and print all entities (ent.text) along with their assigned labels (ent.label_)
# The model identifies specific words or phrases as real-world objects.
[(ent.text, ent.label_) for ent in doc.ents]
# Expected Output (using a standard spaCy model): [('jesus', 'PERSON'), ('galilee', 'LOC')]

Entities:


[('jesus', 'PERSON')]

## Syntactic & Semantic Analysis

 **Syntactic** and **Semantic** analysis are two crucial steps in Natural Language Understanding (NLU) that go beyond simple entity recognition to determine the grammatical structure and the deep meaning of a sentence.

**1. Syntactic Analysis (Dependency Parsing)**

Syntactic analysis, often implemented using Dependency Parsing, focuses on the grammatical structure of the sentence. It determines how the words in a sentence relate to each other by establishing directional, typed links (dependencies) between them:

- **Goal:** To determine the role of each word (e.g., subject, object, modifier) and how it modifies or relates to the main verb (the ROOT).

In [16]:
for token in doc:
    print(token.text, "->", token.dep_, "/", token.pos_)

jesus -> nsubj / PROPN
went -> ROOT / VERB
to -> prep / ADP
galilee -> pobj / PROPN
and -> cc / CCONJ
taught -> conj / VERB
in -> prep / ADP
synagogues -> pobj / NOUN
. -> punct / PUNCT


- Code Explanation (Syntactic Analysis)

The code iterates over the doc object (processed by spaCy) and prints the two main syntactic annotations for each token:

Output Column |	SpaCy Attribute	| Description	 | Example
| :--- | :--- | :--- | :--- |
token.dep_ | **Dependency Label** |	The grammatical role of the token relative to its head (the word it modifies). |	nsubj (Nominal Subject), prep (Prepositional Modifier)
token.pos_ | **Part-of-Speech (POS) Tag** |	The general grammatical category of the token. |	PROPN (Proper Noun), VERB, ADP (Adposition/Preposition)

Analysis of the Output:

The output shows the structural breakdown of the sentence "jesus went to galilee and taught in synagogues.":

Token	| Dependency (dep_) |	POS Tag (pos_) |	Role
| :--- | :--- | :--- | :--- |
jesus |	nsubj (Nominal Subject) |	PROPN |	The subject of the sentence's main action.
went | ROOT |	VERB | The main verb and structural center of the sentence.
to |	prep (Preposition) |	ADP	| The preposition introducing the object of the location.
galilee	| pobj (Object of Preposition) |	PROPN |	The geographical place, governed by "to".
and	| cc (Coordinating Conjunction) |	CCONJ |	Links the two main verbs.
taught |	conj (Conjunct) |	VERB |	The second main action, coordinated with "went".
in |	prep |	ADP |	Preposition introducing the location for teaching.
synagogues |	pobj |	NOUN |	The place of teaching, governed by "in".

**2. Semantic Analysis (Understanding Meaning)**

Semantic analysis focuses on extracting the meaning or contextual intent from the structured output of the syntactic analysis.

- **Goal:** To understand who did what to whom, where, and when.

While the code above primarily shows syntactic analysis, the output is directly used for semantic interpretation:

**Subject/Action**: By identifying jesus as the nsubj (subject) of the ROOT verb went, the system semantically understands: "Jesus performed the action of going."

**Location/Goal:** By linking galilee (pobj) via the preposition to (prep) to the verb went, the system understands: "The destination of the going action was Galilee."

In advanced LLM pipelines, this structured semantic understanding is often used to execute downstream tasks, such as generating code (Code Generation), answering complex questions (Question Answering), or filling knowledge base slots.

### Contextual Disambiguation

**Contextual Disambiguation** in the context of RAG (Retrieval-Augmented Generation) is the critical process of using vector similarity search to retrieve text that is semantically relevant to a query, thereby providing the necessary context for an LLM to generate an accurate, informed answer.

It addresses the fundamental problem that words can have multiple meanings (ambiguity) and that the intended meaning is only clear when considering the surrounding text. The vector embedding model handles this automatically by assigning a unique numerical vector to a phrase or chunk based on its entire context.

In [17]:
query = "Where did Jesus preach?"

query_vec = model.encode([query])
D, I = index.search(query_vec.astype("float32"), k=2)

print("Most relevant chunk:\n")
print(chunks[I[0][0]])

Most relevant chunk:

1:35 and in the morning, rising up a great while before day, he went out, and departed into a solitary place, and there prayed. 1:36 and simon and they that were with him followed after him. 1:37 and when they had found him, they said unto him, all men seek for thee. 1:38 and he said unto them, let us go into the next towns, that i may preach there also: for therefore came i forth. 1:39 and he preached in their synagogues throughout all galilee, and cast out devils. 1:40 and there came a leper to him, beseeching him, and kneeling down to him, and saying unto him, if thou wilt, thou canst make me clean. 1:41 and jesus, moved with compassion, put forth his hand, and touched him, and saith unto him, i will; be thou clean.


#### The Retrieval Mechanism

The provided code demonstrates the final, working step of a basic RAG pipeline: using the vector store to perform an efficient search for context.

1. **Query Vectorization:** The user's question (query) is first converted into a dense vector (query_vec) using the same SentenceTransformer model that was used to embed the document chunks.

2. **Similarity Search:** The FAISS index is queried using this vector. It calculates the numerical distance (e.g., L2 or cosine distance) between the query_vec and every vector stored in the index (emb_matrix).

3. **Top-K Retrieval:** The index.search(..., k=2) command retrieves the indices (I) and distances (D) of the $k$ most similar vectors. In this case, $k=2$, meaning it finds the two most relevant chunks.

#### Code and Output Analysis

Code/Output Section	| Explanation |	Relevance to Disambiguation
| :--- | :--- | :--- |
query = "Where did Jesus preach?"	 | The **user query** intended to find locations of preaching. |	The embedding model must differentiate between "preach" (a verb) and other uses of the word.
D, I = index.search(query_vec.astype("float32"), k=2)	| Executes the FAISS similarity search, returning the distances (D) and the indices (I) of the top 2 matching vectors. |	The search mechanism successfully finds the vector closest to the query's meaning.
print(chunks[I[0][0]]) | Retrieves and prints the text of the most relevant chunk (at index I[0][0]).	| This is the retrieved context that the LLM will use.
Output Chunk | The text contains the exact context needed: "...let us go into the next towns, that i may preach there also... and he preached in their synagogues throughout all galilee..." |	The system successfully disambiguated the query and retrieved a chunk rich with relevant keywords ("preach," "synagogues," "galilee") that are semantically close to the query.

The retrieved chunk is the final evidence that the entire pipeline—from chunking to embedding and searching—has correctly performed contextual disambiguation, providing the precise textual evidence needed to answer the user's question.

### Semantic Similarity Search

**Semantic Similarity Search** is the core mechanism of the Retrieval-Augmented Generation (RAG) pipeline. It utilizes the **vector embeddings** stored in the vector store (FAISS, in this case) **to find the text chunks that are most conceptually similar to a user's query**. This process is essential because it retrieves relevant context based on meaning, not just keyword overlap.

### How the Search Works

The process involves converting the input query into a numerical vector and using that vector to navigate the dense, multi-dimensional embedding space:

1. **Query Vectorization:** The user's input (query) is passed through the **same embedding model** (model.encode) used to create the document embeddings. This generates a high-dimensional vector ($\vec{q}$) representing the query's meaning.

2. **Vector Distance Calculation:** The FAISS index is specialized for calculating the distance between $\vec{q}$ and every document vector ($\vec{d}_i$) stored within it. The distance ($D$) is typically measured using **L2 (Euclidean) distance** or **Cosine Similarity**.

3. **Top-K Retrieval**: The index.search(vec, k) function efficiently finds the $k$ document vectors that are closest to the query vector. These closest vectors represent the document chunks with the most similar meaning.

  - The output $I$ contains the **indices** (locations in the original chunks list) of the top $k$ matches.

  - The output $D$ contains the corresponding **distance scores**.

The function returns the top $k$ text passages along with their distance scores, which quantify the semantic similarity. A lower L2 distance score indicates higher similarity.

In [18]:
def search(query, k=3):
    # Convert the input query string into a float32 vector using the embedding model
    vec = model.encode([query]).astype("float32")

    # Perform the search on the FAISS index:
    # D = Distances (scores), I = Indices (locations of the chunks)
    D, I = index.search(vec, k)

    # Zip the indices and distances together, and return the corresponding chunk text
    # (truncated to 300 characters) and the score.
    return [(chunks[i][:300], float(Di)) for i, Di in zip(I[0], D[0])]

# Execute the search for the specific query: "The birth of Jesus"
results = search("The birth of Jesus")

for passage, score in results:
    print("----")
    print("Score:", score)
    print(passage)

----
Score: 0.9018888473510742
1:1 the book of the generation of jesus christ, the son of david, the son of abraham.
----
Score: 1.015718936920166
2:39 and when they had performed all things according to the law of the lord, they returned into galilee, to their own city nazareth. 2:40 and the child grew, and waxed strong in spirit, filled with wisdom: and the grace of god was upon him. 2:41 now his parents went to jerusalem every year at the f
----
Score: 1.0222469568252563
2:1 now when jesus was born in bethlehem of judaea in the days of herod the king, behold, there came wise men from the east to jerusalem, 2:2 saying, where is he that is born king of the jews? for we have seen his star in the east, and are come to worship him. 2:3 when herod the king had heard these


The output demonstrates the result of the semantic search:

Item	| Description |	Significance |
|:--- |:--- |:--- |
Score:|	The L2 distance between the query vector and the chunk vector. |	A lower score means the chunk is more similar in meaning to the query. |
Passage | The text content of the retrieved chunk. | For the query "The birth of Jesus," the retrieved passages are highly likely to contain verses detailing the nativity scene or the genealogy leading up to the birth.|

This output confirms that your **Semantic Similarity Search** is working correctly. It effectively retrieved the top three chunks most relevant to the query "The birth of Jesus," based on the **L2 distance scores**.

Here is a breakdown of what the scores and retrieved passages mean:

#### Semantic Search Analysis

The function retrieved the following three passages, ordered from most similar (lowest score) to least similar (highest score).


Rank	| Score | (L2 Distance)	Retrieved Passage	Relevance Analysis |
|:--- |:--- |:--- |
1	| 0.9018 (Most Similar)	| 1:1 the book of the generation of jesus christ, the son of david, the son of abraham.	This passage discusses the genealogy ("the generation") of Jesus Christ, which is the necessary prerequisite and context for his birth. This is semantically the closest match because it directly establishes the lineage leading to the nativity. |
2 |	1.0157 |	2:39 and when they had performed all things according to the law of the lord, they returned into galilee, to their own city nazareth. 2:40 and the child grew, and waxed strong in spirit... |	This passage discusses the events immediately following the birth (the return to Nazareth, the growth of the child). While not the act of birth itself, it is highly relevant **post-birth context**.
3 |	1.0222 |	2:1 now when jesus was born in bethlehem of judaea in the days of herod the king, behold, there came wise men from the east to jerusalem, 2:2 saying, where is he that is born king of the jews?...	| This passage contains the most explicit textual reference: "when jesus was born in bethlehem of judaea." However, its lower rank (higher score) suggests the embedding model considered the genealogy (Rank 1) and the post-birth context (Rank 2) slightly more semantically related to the concept of "birth" than this passage, which heavily features the Wise Men and Herod alongside the birth event.

#### Conclusion on Accuracy
The results are accurate for an L2 distance search:

1. **Lower Score = Higher Similarity:** The chunk with the lowest score (0.9018) is considered the closest in the vector space, demonstrating the correct function of the L2 distance metric.

**Contextual Relevance:** All three retrieved chunks are highly relevant to the concept of the birth, encompassing the lineage, the event itself, and the immediate aftermath. This confirms the quality of the chunking and embedding process for RAG.







### Knowledge Retrieval (full RAG query)

The final step in the RAG pipeline combines the retrieval component (FAISS search) with a simulated generation component (a structured prompt). The rag_answer function takes a question, retrieves the most relevant contextual passages from the FAISS index, and then formats them together with the question into a single, cohesive prompt. This prompt is what would typically be sent to a Large Language Model (LLM) to generate a concise, context-aware answer.

This approach ensures the LLM's answer is grounded in the specific data (the Gospel of Matthew) stored in the vector index, mitigating the risk of factual inaccuracies or hallucinations.

In [19]:
import textwrap

def rag_answer(question):
    context = "\n".join([r[0] for r in search(question, k=5)])

    prompt = f"""
    Context:
    {context}

    Question: {question}
    Provide a simple answer using only the context above.
    """

    print(textwrap.fill(prompt, 120))

rag_answer("Who was the mother of Jesus?")

     Context:     27:54 now when the centurion, and they that were with him, watching jesus, saw the earthquake, and
those things that were done, they feared greatly, saying, truly this was the son of god. 27:55 and many women were there
beholding afar off, which followed jesus from galilee, ministering unto him: 27: 3:23 and jesus himself began to be
about thirty years of age, being as was supposed the son of joseph, which was the son of heli, 3:24 which was the son of
matthat, which was the son of levi, which was the son of melchi, which was the son of janna, which was the son of
joseph, 3:25 which was the son 15:38 and the veil of the temple was rent in twain from the top to the bottom. 15:39 and
when the centurion, which stood over against him, saw that he so cried out, and gave up the ghost, he said, truly this
man was the son of god. 15:40 there were also women looking on afar off: among whom was mary 2:34 and simeon blessed
them, and said unto mary his mother, behold, this child

The rag_answer function structure is as follows:

1. Retrieval: It uses the pre-defined search(question, k=5) function to query the FAISS index. This returns the text of the top $k=5$ most semantically similar chunks (passages) related to the question.

In [20]:
print(search("Who was the mother of Jesus?", k=5))

[('27:54 now when the centurion, and they that were with him, watching jesus, saw the earthquake, and those things that were done, they feared greatly, saying, truly this was the son of god. 27:55 and many women were there beholding afar off, which followed jesus from galilee, ministering unto him: 27:', 0.8386414051055908), ('3:23 and jesus himself began to be about thirty years of age, being as was supposed the son of joseph, which was the son of heli, 3:24 which was the son of matthat, which was the son of levi, which was the son of melchi, which was the son of janna, which was the son of joseph, 3:25 which was the son', 0.8883230686187744), ('15:38 and the veil of the temple was rent in twain from the top to the bottom. 15:39 and when the centurion, which stood over against him, saw that he so cried out, and gave up the ghost, he said, truly this man was the son of god. 15:40 there were also women looking on afar off: among whom was mary', 0.9170591831207275), ('2:34 and simeon ble

2. Context Aggregation: It joins the retrieved passages into a single context string, separated by newlines.

3. Prompt Construction: It builds a final prompt template containing three distinct parts:

- The complete Context (the retrieved passages).
- The original Question.
- A simple Instruction to the LLM: "Provide a simple answer using only the context above."

4. Output: It prints the final, formatted prompt using textwrap.fill for clean, line-wrapped display.

#### Code Explanation and Output

The provided code defines the rag_answer function to execute the full RAG process.

- context = "\n".join([r[0] for r in search(question, k=5)]): Executes the vector search for the question. It retrieves the top 5 relevant chunks (k=5) and joins only the text content (r[0]) into a single string.

- The prompt is a f-string that formats the retrieved context, the user's question, and a clear instruction into the structure designed for an LLM.

- print(textwrap.fill(prompt, 120)): Prints the final prompt, wrapping lines at 120 characters for readability.

**Output for** rag_answer("Who was the mother of Jesus?")

The FAISS search retrieves chunks that mention Jesus, Mary, or related names, leading to this output:

>Context:     27:54 now when the centurion, and they that were with him, watching jesus, saw the earthquake, and
those things that were done, they feared greatly, saying, truly this was the son of god. 27:55 and many women were there
beholding afar off, which followed jesus from galilee, ministering unto him: 27: 3:23 and jesus himself began to be
about thirty years of age, being as was supposed the son of joseph, which was the son of heli, 3:24 which was the son of
matthat, which was the son of levi, which was the son of melchi, which was the son of janna, which was the son of
joseph, 3:25 which was the son 15:38 and the veil of the temple was rent in twain from the top to the bottom. 15:39 and
when the centurion, which stood over against him, saw that he so cried out, and gave up the ghost, he said, truly this
man was the son of god. 15:40 there were also women looking on afar off: among whom was mary 2:34 and simeon blessed
them, and said unto mary his mother, behold, this child is set for the fall and rising again of many in israel; and for
a sign which shall be spoken against; 2:35 yea, a sword shall pierce through thy own soul also, that the thoughts of
many hearts may be revealed. 2:36 and t 1:21 and she shall bring forth a son, and thou shalt call his name jesus: for he
shall save his people from their sins. 1:22 now all this was done, that it might be fulfilled which was spoken of the
lord by the prophet, saying, 1:23 behold, a virgin shall be with child, and shall bring forth a son,       Question: Who
was the mother of Jesus?     Provide a simple answer using only the context above.

**The key piece of context retrieved is:**
>2:34 and simeon blessed them, and said unto mary his mother, behold, this child is set for the fall and rising again of many in israel;

which directly identifies **Mary** as his mother. An LLM would use this specific line to answer the question.

### GENERATING TEXT

You are now at the stage where the Retrieval-Augmented Generation (RAG) system would take the final structured prompt (including the retrieved context and the plan) and send it to a Large Language Model (LLM) to produce the final summary.

In [21]:
plan = {
    "topic": "Summary of Matthew Chapter 5",
    "key_points": [
        "Beatitudes",
        "Salt and light",
        "Law fulfillment",
        "Anger, lust, divorce, oaths"
    ]
}

plan

{'topic': 'Summary of Matthew Chapter 5',
 'key_points': ['Beatitudes',
  'Salt and light',
  'Law fulfillment',
  'Anger, lust, divorce, oaths']}

#### Code Explanation

The provided code block simply defines a Python dictionary named plan and then displays its content. This dictionary is an example of a structured input plan that could be used to guide a generative model (like an LLM) in creating a specific piece of text, such as a summary of a Bible chapter.

```
plan = {
    "topic": "Summary of Matthew Chapter 5",
    "key_points": [
        "Beatitudes",
        "Salt and light",
        "Law fulfillment",
        "Anger, lust, divorce, oaths"
    ]
}

plan
```

- Dictionary Definition: The code initializes a variable called plan and assigns it a dictionary ({...}). Dictionaries are used to store data in key-value pairs.

- Keys and Values:

  - The key "topic" is assigned the string value "Summary of Matthew Chapter 5". This defines the overall subject of the text to be generated.

  - The key "key_points" is assigned a list of strings ([...]). This list serves as a set of required sub-headings or mandatory inclusions that the generated text must cover.

- **Output:** The final line, plan, is not an explicit print() statement, but in an interactive Python environment (like a Jupyter Notebook or the one used in the user's context), simply typing the variable name will cause the notebook to output the variable's final value.

#### Purpose in a RAG/Generation Pipeline

In a full Retrieval-Augmented Generation (RAG) or text generation pipeline, this dictionary serves as the final structured input that complements the retrieved context.

1. Retrieval (RAG): The system first retrieves relevant passages (context) from the vector database (FAISS index) based on the overall "topic".

2. Generation: This plan dictionary is then often converted into a detailed prompt (e.g., using JSON formatting or a structured list) and passed to a Large Language Model (LLM) along with the retrieved context.

  - The prompt would instruct the LLM: "Using the following context, write a summary on the topic and ensure you address all the key_points."

This process ensures the generated text is accurate (grounded in the retrieved context) and comprehensive (structured according to the predefined plan).
